In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
inpatient_specialty = pd.read_csv('/data/public/MLA/VCHAMPS-Test/inpatient_specialty_test.csv').iloc[:,1:]

In [3]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [4]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [5]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

In [6]:
path = '/home/hassan/lily/MLA/FDA/inpatient_simple_test.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.head()

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,Died,AO,CVD,unique_admitting_specialty,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,7,5,44.40,3,0,0,0,1,0,4,...,0.50,2,1,0,2,0,0,338.72,2013-04-18 14:41:31.0,0
1,9,4,10.70,2,0,0,0,0,0,3,...,4.00,0,0,0,4,0,0,2.03,2000-04-10 02:38:04.0,1
2,12,5,21.54,1,0,4,0,1,1,2,...,1.25,5,0,0,0,0,1,69.25,2004-04-07 03:40:16.0,1
3,17,12,46.45,2,0,5,0,0,1,4,...,3.00,0,1,0,11,0,0,61.89,2004-05-05 15:18:05.0,1
4,22,2,4.53,2,1,0,0,0,0,2,...,0.17,0,0,0,2,0,0,0.42,2014-05-07 15:43:10.0,1


In [7]:
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date

,Internalpatientid,Discharge date
0,7,2013-04-18 14:41:31.0
1,9,2000-04-10 02:38:04.0
2,12,2004-04-07 03:40:16.0
3,17,2004-05-05 15:18:05.0
4,22,2014-05-07 15:43:10.0
...,...,...
22250,168995,2010-06-09 10:03:21.0
22251,169011,2004-07-15 15:11:27.0
22252,169037,2015-02-12 00:48:40.0
22253,169059,2003-03-11 12:21:08.0


In [8]:
inpatient_specialty2 = inpatient_specialty.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
inpatient_specialty3 = inpatient_specialty2.loc[inpatient_specialty2['Specialty end date'] <= inpatient_specialty2['Discharge date']]
inpatient_specialty3

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded,Discharge date
0,100004,81.261813,2014-04-06 04:37:21,2014-04-08 06:17:29,CARDIAC-STEP DOWN UNIT,Florida,2 days 01:40:08,0.0,2014-04-08 06:17:30.0
2,100035,52.316254,2014-07-13 14:10:34,2014-07-15 16:10:05,BLIND REHAB OBSERVATION,District of Columbia,2 days 01:59:31,1.0,2016-11-02 14:58:44.0
3,100041,89.583528,2014-11-05 18:42:42,2014-11-06 04:08:18,NH SHORT-STAY CONTINUING CARE,Minnesota,0 days 09:25:36,1.0,2022-09-20 02:09:41.0
4,100041,89.595286,2014-11-10 01:51:04,2014-11-10 01:51:04,ORTHOPEDIC,Minnesota,0 days 00:00:00,0.0,2022-09-20 02:09:41.0
5,100041,90.121030,2015-05-21 05:40:21,2015-05-21 05:40:21,NH SHORT STAY REHABILITATION,Minnesota,0 days 00:00:00,1.0,2022-09-20 02:09:41.0
...,...,...,...,...,...,...,...,...,...
386891,9996,77.010416,2005-12-21 07:22:13,2005-12-22 03:51:27,GEM REHABILITATION MEDICINE,Pennsylvania,0 days 20:29:14,1.0,2013-04-09 19:31:55.0
386892,9996,84.296993,2013-04-06 13:13:12,2013-04-08 17:12:56,GEM INTERMEDIATE CARE,Pennsylvania,2 days 03:59:44,0.0,2013-04-09 19:31:55.0
386893,99964,73.451708,2016-07-30 14:22:55,2016-07-30 14:22:55,GENERAL(ACUTE MEDICINE),Oregon,0 days 00:00:00,0.0,2021-03-06 09:39:30.0
386894,99989,71.251809,2003-10-03 12:52:03,2003-10-04 04:18:52,DOD BEDS IN VA FACILITY,Ohio,0 days 15:26:49,1.0,2011-08-03 21:30:06.0


In [9]:
inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('count')

specialty_medical_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 0).sum()).reset_index(name='specialty medical count')

specialty_support_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 1).sum()).reset_index(name='specialty support count')

inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('mean')

/tmp/ipykernel_244617/3919990082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
/tmp/ipykernel_244617/3919990082.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\


In [10]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

/tmp/ipykernel_244617/1693156475.py:2: RuntimeWarning: Mean of empty slice
  period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [11]:
inpatient_specialty4 = inpatient_specialty3[['Internalpatientid', 'mean age at specailty','specialty count']].sort_values('Internalpatientid')
inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)
inpatient_specialty4 = inpatient_specialty4.drop_duplicates().reset_index(drop = True)
inpatient_specialty4

/tmp/ipykernel_244617/554132651.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,mean age at specailty,specialty count
0,7,62.000986,11
1,9,51.712472,7
2,12,72.135061,10
3,17,80.727176,26
4,22,54.075185,3
...,...,...,...
22208,168995,63.365209,4
22209,169011,60.826909,2
22210,169037,83.911854,3
22211,169059,79.905892,1


In [12]:
inpatient_specialty5 = inpatient_specialty4.merge(period_mean, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(period_std, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.merge(specialty_medical_count, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(specialty_support_count, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.dropna(subset = 'period mean')
inpatient_specialty5

,Internalpatientid,mean age at specailty,specialty count,period mean,period std,specialty medical count,specialty support count
0,7,62.000986,11,3.579801,7.603465,7,4
1,9,51.712472,7,1.945340,2.069379,5,2
2,12,72.135061,10,2.031376,2.952513,10,0
3,17,80.727176,26,1.622804,2.126398,11,15
4,22,54.075185,3,1.466395,2.058525,1,2
...,...,...,...,...,...,...,...
22208,168995,63.365209,4,0.668522,0.750610,3,1
22209,169011,60.826909,2,1.668035,2.455277,2,0
22210,169037,83.911854,3,11.138164,19.410162,3,0
22211,169059,79.905892,1,4.455550,4.455550,0,1


In [13]:
inpatient_specialty5.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_preprocessed_test.csv')